<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# 데이터 스트림을 처리하는 대화형 LLM 에이전트

AutoGen은 자동화된 채팅을 통해 공동으로 작업을 수행하는 데 사용할 수 있는 LLM, 도구 또는 사람으로 구동되는 대화형 에이전트를 제공합니다. 이 프레임워크는 다중 에이전트 대화를 통해 툴 사용과 사람 참여를 허용합니다.
이 기능에 대한 설명서는 [여기](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)에서 확인할 수 있습니다.

이 노트북에서는 맞춤형 에이전트를 사용하여 웹에서 지속적으로 뉴스를 수집하고 투자 제안을 요청하는 방법을 보여드립니다.

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 설치하세요:
```bash
pip 설치 pyautogen
```

In [1]:
# %pip install pyautogen~=0.1.0

## API 엔드포인트 설정

`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다.


In [2]:
import autogen

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

먼저 유효한 json 문자열이어야 하는 환경 변수 "OAI_CONFIG_LIST"를 찾습니다. 해당 변수를 찾을 수 없으면 "OAI_CONFIG_LIST"라는 이름의 json 파일을 찾습니다. 이 파일은 모델별로 구성을 필터링합니다(다른 키로도 필터링 가능). 필터 조건에 따라 이름이 일치하는 모델만 목록에 유지됩니다.

구성 목록은 다음과 같습니다:
```python
config_list = [
    {
        '모델': 'gpt-4',
        'api_key': '<여기에 OpenAI API 키 입력>',
    }, # gpt-4용 OpenAI API 엔드포인트
    {
        'model': 'gpt-4',
        'api_key': '<여기에 첫 번째 Azure OpenAI API 키>',
        'api_base': '<여기에 첫 번째 Azure OpenAI API 베이스>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # gpt-4용 Azure OpenAI API 엔드포인트
    {
        'model': 'gpt-4',
        'api_key': '<여기에 두 번째 Azure OpenAI API 키>',
        'api_base': '<두 번째 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # gpt-4에 대한 또 다른 Azure OpenAI API 엔드포인트
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<여기에 OpenAI API 키>',
    }, # gpt-3.5-turbo용 OpenAI API 엔드포인트
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<여기에 첫 번째 Azure OpenAI API 키>',
        'api_base': '<여기에 첫 번째 Azure OpenAI API 기반>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # gpt-3.5-turbo용 Azure OpenAI API 엔드포인트
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<여기에 두 번째 Azure OpenAI API 키>',
        'api_base': '<두 번째 Azure OpenAI API 기반은 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # gpt-3.5-turbo에 대한 또 다른 Azure OpenAI API 엔드포인트
]
```

콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 "파일 업로드" 아이콘을 선택해 파일을 업로드할 수 있습니다.

YAML 파일에서 불러오기 등 원하는 다른 방법으로 config_list의 값을 설정할 수 있습니다.

## 예제 작업: 실시간 데이터를 통한 투자 제안

소스에서 뉴스 데이터를 스트리밍하고 어시스턴트 에이전트를 사용하여 데이터를 기반으로 지속적으로 투자 제안을 제공하는 시나리오를 고려해 보겠습니다.

먼저 다음 코드를 사용하여 데이터 스트림 프로세스를 시뮬레이션합니다.

In [3]:
import asyncio

def get_market_news(ind, ind_upper):
    import requests
    import json
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&sort=LATEST&limit=5&apikey=demo'
    # r = requests.get(url)
    # data = r.json()
    # with open('market_news_local.json', 'r') as file:
    #     # Load JSON data from file
    #     data = json.load(file)
    data = {
        "feed": [
            {
                "title": "Palantir CEO Says Our Generation's Atomic Bomb Could Be AI Weapon - And Arrive Sooner Than You Think - Palantir Technologies  ( NYSE:PLTR ) ",
                "summary": "Christopher Nolan's blockbuster movie \"Oppenheimer\" has reignited the public discourse surrounding the United States' use of an atomic bomb on Japan at the end of World War II.",
                "overall_sentiment_score": 0.009687,
            },
            {
                "title": '3 "Hedge Fund Hotels" Pulling into Support',
                "summary": "Institutional quality stocks have several benefits including high-liquidity, low beta, and a long runway. Strategist Andrew Rocco breaks down what investors should look for and pitches 3 ideas.",
                "banner_image": "https://staticx-tuner.zacks.com/images/articles/main/92/87.jpg",
                "overall_sentiment_score": 0.219747,
            },
            {
                "title": "PDFgear, Bringing a Completely-Free PDF Text Editing Feature",
                "summary": "LOS ANGELES, July 26, 2023 /PRNewswire/ -- PDFgear, a leading provider of PDF solutions, announced a piece of exciting news for everyone who works extensively with PDF documents.",
                "overall_sentiment_score": 0.360071,
            },
            {
                "title": "Researchers Pitch 'Immunizing' Images Against Deepfake Manipulation",
                "summary": "A team at MIT says injecting tiny disruptive bits of code can cause distorted deepfake images.",
                "overall_sentiment_score": -0.026894,
            },
            {
                "title": "Nvidia wins again - plus two more takeaways from this week's mega-cap earnings",
                "summary": "We made some key conclusions combing through quarterly results for Microsoft and Alphabet and listening to their conference calls with investors.",
                "overall_sentiment_score": 0.235177,
            },
        ]
    }
    feeds = data["feed"][ind:ind_upper]
    feeds_summary = "\n".join(
        [
            f"News summary: {f['title']}. {f['summary']} overall_sentiment_score: {f['overall_sentiment_score']}"
            for f in feeds
        ]
    )
    return feeds_summary

data = asyncio.Future()

async def add_stock_price_data():
    # simulating the data stream
    for i in range(0, 5, 1):
        latest_news = get_market_news(i, i + 1)
        if data.done():
            data.result().append(latest_news)
        else:
            data.set_result([latest_news])
        # print(data.result())
        await asyncio.sleep(5)

data_task = asyncio.create_task(add_stock_price_data())


그런 다음 에이전트를 구축합니다. LLM을 사용하여 질문에 답변하기 위해 보조 에이전트가 생성됩니다. 사용자 프록시 에이전트가 생성되어 질문을 하고 사용 가능한 새 데이터가 있으면 대화에 추가합니다.

In [4]:
import autogen

# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "request_timeout": 600,
        "seed": 41,
        "config_list": config_list,
        "temperature": 0,
    },
    system_message="You are a financial expert.",
)
# create a UserProxyAgent instance named "user"
user_proxy = autogen.UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config=False,
    default_auto_reply=None,
)

async def add_data_reply(recipient, messages, sender, config):
    await asyncio.sleep(0.1)
    data = config["news_stream"]
    if data.done():
        result = data.result()
        if result:
            news_str = "\n".join(result)
            result.clear()
            return (
                True,
                f"Just got some latest market news. Merge your new suggestion with previous ones.\n{news_str}",
            )
        return False, None

user_proxy.register_reply(autogen.AssistantAgent, add_data_reply, 1, config={"news_stream": data})

사용자 프록시 에이전트의 `a_initiate_chat()` 메서드를 호출하여 대화를 시작합니다.

In [5]:
await user_proxy.a_initiate_chat(
    assistant,
    message="""Give me investment suggestion in 3 bullet points.""",
)
while not data_task.done() and not data_task.cancelled():
    reply = await user_proxy.a_generate_reply(sender=assistant)
    if reply is not None:
        await user_proxy.a_send(reply, assistant)

user (to assistant):

Give me investment suggestion in 3 bullet points.

--------------------------------------------------------------------------------
assistant (to user):

1. Diversify Your Portfolio: Don't put all your eggs in one basket. Spread your investments across a variety of asset classes such as stocks, bonds, real estate, and commodities. This can help to mitigate risk and potentially increase returns.

2. Invest for the Long Term: Investing is not about making a quick buck, but about growing your wealth over time. Stick to a long-term investment strategy and avoid the temptation to engage in frequent buying and selling.

3. Regularly Review Your Investments: The market is dynamic and constantly changing. Regularly review your investment portfolio to ensure it aligns with your financial goals and risk tolerance. Adjust your investments as necessary based on changes in your personal circumstances and market conditions.

-----------------------------------------------------